In [1]:
import os
import datetime
import matplotlib.pylab as plt
from matplotlib.patches import Ellipse
import numpy as np
from IPython.display import display, Math, Latex
import csv
import pandas as pd
import seaborn as sns
from dateutil import tz
from scipy.optimize import curve_fit
import collections

import netCDF4 as nc
from salishsea_tools import (viz_tools, tidetools as tt, ellipse, nc_tools)
from salishsea_tools.nowcast import (analyze, research_VENUS)


%matplotlib inline

In [2]:
path = '/data/dlatorne/MEOPAR/SalishSea/nowcast/'

to=datetime.datetime(2015,4,26)
tf=datetime.datetime(2015,7,19)

i = [280]
j = [400]

In [3]:
params, dep = ellipse.get_params_nowcast(to, tf, i, j, path, 4, depthrange=1, depav=False)
print params['M2']['Phase'], params['K1']['Semi-Major Axis'], 

[[ 333.9650171]] [[ 0.00215212]]


In [4]:
params, dep = ellipse.get_params_nowcast(to, tf, i, j, path, 6, depthrange=1, depav=False)
print params['M2']['Phase'], params['P1']['Semi-Major Axis']

[[ 334.25039193]] [[ 0.00123092]]


In [5]:
params, dep = ellipse.get_params_nowcast(to, tf, i, j, path, 8, depthrange=1, depav=False)
print params['M2']['Phase'], params['P1']['Semi-Major Axis']

[[ 334.38448703]] [[ 0.00122697]]
